# Linear solve

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

np.random.seed(12)
starry.config.lazy = False
starry.config.quiet = True

Instantiate a reflected light map of the Earth:

In [ ]:
map = starry.Map(ydeg=10, reflected=True)
map.inc = 60
map.load("earth")
map.show(projection="rect", illuminate=False)

Generate a dataset:

In [ ]:
# Make the planet rotate 10 times over one full orbit
npts = 10000
nrot = 10
time = np.linspace(0, 1, npts)
theta = np.linspace(0, 360 * nrot, npts)

# Position of the star relative to the planet in the orbital plane
t = np.reshape(time, (1, -1))
p = np.vstack((np.cos(2 * np.pi * t), np.sin(2 * np.pi * t), 0 * t))

# Rotate to an observer inclination of 60 degrees
ci = np.cos(60 * np.pi / 180)
si = np.sin(60 * np.pi / 180)
R = np.array([[1, 0, 0], [0, ci, -si], [0, si, ci]])
xs, ys, zs = R.dot(p)

# Keywords to the `flux` method
kwargs = dict(theta=theta, xs=xs, ys=ys, zs=zs)

In [ ]:
# Compute the flux
flux0 = map.flux(**kwargs)
sigma = 0.01
flux = flux0 + sigma * np.random.randn(npts)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.plot(time, flux)
ax.set_xlabel("Orbital phase", fontsize=18)
ax.set_ylabel("Normalized flux", fontsize=18);

Now the fun part. Let's instantiate a new map so we can do inference on this dataset:

In [ ]:
map = starry.Map(ydeg=10, reflected=True)
map.inc = 60

We now set the data vector (the flux and the covariance matrix) and the prior (`L` is the prior variance of the spherical harmonic coefficients, which we set to something small):

In [ ]:
map.set_data(flux, C=sigma ** 2)
map.set_prior(L=1e-3)

Finally, we call `solve`, passing in the `kwargs` from before. In this case, we're assuming we know the orbital information exactly. (When this is not the case, we need to do sampling for the orbital parameters; we cover this in more detail in the **Eclipsing Binary** tutorial).

In [ ]:
%%time
yhat, cho_ycov = map.solve(**kwargs)

We can now draw a sample from the posterior (and set the map coefficients accordingly) by calling

In [ ]:
map.draw()

That's it! Let's look at our sample map:

In [ ]:
map.show(projection="rect", illuminate=False)

We can verify that we got a good fit to the data:

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.plot(time, flux)
plt.plot(time, map.flux(**kwargs))
ax.set_xlabel("Orbital phase", fontsize=18)
ax.set_ylabel("Normalized flux", fontsize=18);